<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#More-examples" data-toc-modified-id="More-examples-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>More examples</a></span><ul class="toc-item"><li><span><a href="#Computing-matches-Euclidean-similarity" data-toc-modified-id="Computing-matches-Euclidean-similarity-0.1.1"><span class="toc-item-num">0.1.1&nbsp;&nbsp;</span>Computing matches Euclidean similarity</a></span></li><li><span><a href="#Computing-matches-Euclidean-distance" data-toc-modified-id="Computing-matches-Euclidean-distance-0.1.2"><span class="toc-item-num">0.1.2&nbsp;&nbsp;</span>Computing matches Euclidean distance</a></span></li><li><span><a href="#Computing-matches-Cosine-similarity" data-toc-modified-id="Computing-matches-Cosine-similarity-0.1.3"><span class="toc-item-num">0.1.3&nbsp;&nbsp;</span>Computing matches Cosine similarity</a></span></li></ul></li><li><span><a href="#test-distances-with-scipy" data-toc-modified-id="test-distances-with-scipy-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>test distances with scipy</a></span></li><li><span><a href="#Document-Memmap" data-toc-modified-id="Document-Memmap-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Document Memmap</a></span><ul class="toc-item"><li><span><a href="#Test-distances-using-memmaparrays" data-toc-modified-id="Test-distances-using-memmaparrays-0.3.1"><span class="toc-item-num">0.3.1&nbsp;&nbsp;</span>Test distances using memmaparrays</a></span></li></ul></li></ul></li><li><span><a href="#Batching-distance-computation" data-toc-modified-id="Batching-distance-computation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Batching distance computation</a></span></li></ul></div>

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from jina import Document, DocumentArray
import numpy as np

d1 = Document(embedding=np.array([0,0,0,0,1]))
d2 = Document(embedding=np.array([1,0,0,0,0]))
d3 = Document(embedding=np.array([1,1,1,1,0]))
d4 = Document(embedding=np.array([1,2,2,1,0]))

d1_m = Document(embedding=np.array([0,0.1,0,0,0]))
d2_m = Document(embedding=np.array([1,0.1,0,0,0]))
d3_m = Document(embedding=np.array([1,1.2,1,1,0]))
d4_m = Document(embedding=np.array([1,2.2,2,1,0]))
d5_m = Document(embedding=np.array([4,5.2,2,1,0]))

da_1 = DocumentArray([d1, d2, d3, d4])
da_2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])


da_1.match(da_2, metric='euclidean',  limit=3)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', m.embedding, 'score =', m.scores['euclidean'].value)

query emb = [1 1 1 1 0]
match emb = [1.  1.2 1.  1.  0. ] score = 0.20000000298023224
match emb = [1.  2.2 2.  1.  0. ] score = 1.5620499849319458
match emb = [1.  0.1 0.  0.  0. ] score = 1.6763054132461548


In [8]:
from jina import Document, DocumentArray
import scipy.sparse as sp

d1 = Document(embedding=sp.csr_matrix([0,0,0,0,1]))
d2 = Document(embedding=sp.csr_matrix([1,0,0,0,0]))
d3 = Document(embedding=sp.csr_matrix([1,1,1,1,0]))
d4 = Document(embedding=sp.csr_matrix([1,2,2,1,0]))

d1_m = Document(embedding=sp.csr_matrix([0,0.1,0,0,0]))
d2_m = Document(embedding=sp.csr_matrix([1,0.1,0,0,0]))
d3_m = Document(embedding=sp.csr_matrix([1,1.2,1,1,0]))
d4_m = Document(embedding=sp.csr_matrix([1,2.2,2,1,0]))
d5_m = Document(embedding=sp.csr_matrix([4,5.2,2,1,0]))

da_1 = DocumentArray([d1, d2, d3, d4])
da_2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])

da_1.match(da_2, metric='euclidean', limit=3)
query = da_1[2]
print(f'query emb = {query.embedding.todense()}')
for m in query.matches:
    print('match emb =', m.embedding.todense(), 'score =', m.scores['euclidean'].value)

TypeError: matrix is not square

## More examples

In [9]:
from jina import Document, DocumentArray
import numpy as np
from jina.math.distance import cosine, sqeuclidean, cdist

In [10]:
n_features = 100

def get_document_arrays():
    d1 = Document(embedding=np.array([0,0,0,0,0]))
    d2 = Document(embedding=np.array([1,0,0,0,0]))
    d3 = Document(embedding=np.array([1,1,1,1,0]))
    d4 = Document(embedding=np.array([1,2,2,1,0]))

    d1_m = Document(embedding=np.array([0,0.1,0,0,0]))
    d2_m = Document(embedding=np.array([1,0.1,0,0,0]))
    d3_m = Document(embedding=np.array([1,1.2,1,1,0]))
    d4_m = Document(embedding=np.array([1,2.2,2,1,0]))
    d5_m = Document(embedding=np.array([4,5.2,2,1,0]))

    D  = DocumentArray([d1,d2,d3,d4])
    D2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])
    return D, D2

import scipy.sparse as sp

def get_document_arrays_sparse():
    d1 = Document(embedding=sp.csr_matrix([0,0,0,0,0]))
    d2 = Document(embedding=sp.csr_matrix([1,0,0,0,0]))
    d3 = Document(embedding=sp.csr_matrix([1,1,1,1,0]))
    d4 = Document(embedding=sp.csr_matrix([1,2,2,1,0]))

    d1_m = Document(embedding=sp.csr_matrix([0,0.1,0,0,0]))
    d2_m = Document(embedding=sp.csr_matrix([1,0.1,0,0,0]))
    d3_m = Document(embedding=sp.csr_matrix([1,1.2,1,1,0]))
    d4_m = Document(embedding=sp.csr_matrix([1,2.2,2,1,0]))
    d5_m = Document(embedding=sp.csr_matrix([4,5.2,2,1,0]))

    D  = DocumentArray([d1,d2,d3,d4])
    D2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])
    return D, D2

In [11]:
D, D2 = get_document_arrays()

In [12]:
E = np.stack(D.get_attributes('embedding'))
E.argsort(axis=1)

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 0],
       [4, 0, 1, 2, 3],
       [4, 0, 3, 1, 2]])

#### _compute_dist Euclidean distance

In [13]:
X = np.stack(D.get_attributes('embedding'))
Y = np.stack(D2.get_attributes('embedding'))

dists = cdist(X, Y, 'euclidean')
dists

array([[0.1       , 1.00498756, 2.10713075, 3.29241553, 6.93108938],
       [1.00498756, 0.1       , 1.8547237 , 3.13687743, 6.40624695],
       [1.95192213, 1.67630546, 0.2       , 1.56204994, 5.25737577],
       [3.1       , 2.93428015, 1.28062485, 0.2       , 4.38634244]])

### Computing matches Euclidean similarity

By default `D.match` uses a similarity metric.

This means matched results are ordered from highest to lowest

In [14]:
D, D2 = get_document_arrays()
D.match(D2, metric='euclidean', limit=3)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'score =', m.scores['euclidean'].value)

In [ ]:
print(f'query={D[1].embedding}')
for m in D[1].matches:
    print(m.embedding, 'score =', m.scores['euclidean'].value)

In [ ]:
from jina.enums import EmbeddingClsType

D, D2 = get_document_arrays_sparse()
D.match(D2, 
        metric='euclidean',
        limit=3,
        is_sparse=True)
        #embedding_cls_type = EmbeddingClsType.SCIPY_CSR)

In [ ]:
print(f'query={D[1].embedding.todense()}')
for m in D[1].matches:
    print(m.embedding.todense(), 'score =', m.scores['euclidean'].value)

In [ ]:
print(f'query={D[2].embedding.todense()}')
for m in D[2].matches:
    print(m.embedding.todense(), 'score =', m.scores['euclidean'].value)

### Computing matches Euclidean distance

If we want to use a distance we need to specify **`is_distance=True`**.

If we do so we should get results ordered from lowest to highest

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='euclidean', limit=3, is_distance=True)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'distance =', m.scores['euclidean'].value)

### Computing matches Cosine similarity

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='cosine', limit=3)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'dist=', m.scores['cosine'].value)

In [ ]:
scores = [m.scores['cosine'].value for m in D[2].matches]
scores ==sorted(scores)[::-1]

If we want cosine distance we need to specify `is_distance=True`

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='cosine', limit=3, is_distance=True)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'dist=', m.scores['cosine'].value)

In [ ]:
distances = [m.scores['cosine'].value for m in D[2].matches]
distances ==sorted(distances)

In [ ]:
[m.scores['cosine'].value for d in D for m in d.matches]

## test distances with scipy

In [ ]:

from scipy.spatial.distance import cdist

In [ ]:
#X = np.random.random((10,5))
#Y = np.random.random((15,5))

X = np.array([[1,1,1],[4,5,6],[0,1,2]])
Y = np.array([[1,1,2],[2,3,4]])
XYdistances = cdist(X,Y,metric='euclidean')
XY_new = np.sqrt(euclidean_distance_squared(X,Y))
np.testing.assert_almost_equal(XYdistances,XY_new)

In [ ]:
XYdistances = cdist(X,Y,metric='cosine')
XY_new = cosine_distance(X,Y)
np.testing.assert_almost_equal(XYdistances, XY_new)

In [ ]:
XY_new

In [ ]:
XYdistances

In [ ]:
X

In [ ]:
np.linalg.norm(X, axis=1)

In [ ]:
np.linalg.norm(Y, axis=1)

In [ ]:
np.outer(np.linalg.norm(X, axis=1), np.linalg.norm(Y, axis=1))  

In [ ]:
np.outer(np.array([1,2,3]), np.array([3,4]))

In [ ]:
2.44948974 * 1.73205081

## Document Memmap

In [ ]:
from jina.types.arrays.memmap import DocumentArrayMemmap

##### test_match_docarray_docarraymemmap

In [ ]:
metric = 'euclidean'
D1, D2 = get_document_arrays()
D1.match(D2, metric=metric, limit=3, is_distance=True)
values_docarray = [m.scores[metric].value for d in D1 for m in d.matches]

!rm -rf ./my-memmap
D1, D2 = get_document_arrays()
D2memmap = DocumentArrayMemmap('./my-memmap')
D2memmap.extend(D2)
D1.match(D2memmap, metric=metric, limit=3, is_distance=True)
values_docarraymemmap = [m.scores[metric].value for d in D1 for m in d.matches]

In [ ]:
values_docarraymemmap == values_docarraymemmap

##### test_docarraymemmap_match_docarray

In [ ]:
D1, D2 = get_document_arrays()
D1.match(D2, metric='euclidean', limit=3, is_distance=True)
values_docarray = [m.scores['euclidean'].value for d in D1 for m in d.matches]

!rm -rf ./my-memmap

from jina import Document
D1, D2 = get_document_arrays()
D1memmap = DocumentArrayMemmap('./my-memmap')
D1memmap.extend(D1)
D1memmap.match(D2, metric='euclidean', limit=3, is_distance=True)

print(f'query={dam[2].embedding}')
for m in D1memmap[2].matches:
    print(m.embedding, 'distance =', m.scores['euclidean'].value)
    
values_docarraymemmap = [m.scores['euclidean'].value for d in D1memmap for m in d.matches]

In [ ]:
values_docarray == values_docarraymemmap

##### test_docarraymemmap_match_docarraymemmap

In [ ]:
metric = 'euclidean'
is_distance = 'False'
D1, D2 = get_document_arrays()
D1.match(D2, metric=metric, limit=3, is_distance=True)
values_docarray = [m.scores[metric].value for d in D1 for m in d.matches]

!rm -rf ./my-memmap1
!rm -rf ./my-memmap2
D1, D2 = get_document_arrays()
D1memmap = DocumentArrayMemmap('./my-memmap1')
D1memmap.extend(D1)
D1memmap.reload()
D2memmap = DocumentArrayMemmap('./my-memmap2')
D2memmap.extend(D2)
D2memmap.reload()
D2memmap.reload()
D1memmap.match(D2memmap, metric=metric, limit=3, is_distance=is_distance)

#values_docarraymemmap = [m.scores[metric].value for d in D1memmap for m in d.matches]


In [ ]:
values_docarraymemmap = [m.scores[metric].value for d in D1memmap for m in d.matches]
values_docarraymemmap

In [ ]:
D1memmap[0].matches

In [ ]:
values_docarray

### Test distances using memmaparrays

In [15]:
euclidean_distance_squared(D1memmap, D2memmap)

NameError: name 'euclidean_distance_squared' is not defined

In [16]:
D1memmap.get_attributes('embedding')

NameError: name 'D1memmap' is not defined

# Batching distance computation

In [268]:
n_features = 10
np.random.seed(1234)
X = np.random.random((10, n_features))
Y = np.random.random((1000,n_features))

In [270]:
X[0]

array([0.19151945, 0.62210877, 0.43772774, 0.78535858, 0.77997581,
       0.27259261, 0.27646426, 0.80187218, 0.95813935, 0.87593263])

In [279]:
np.argpartition([4,0,2,6,9,1],kth=3)

array([1, 5, 2, 0, 3, 4])

In [78]:
import jina.math 
from jina.math.distance import sqeuclidean

In [79]:
sqeuclidean(X,Y).shape

(10, 1000)

In [80]:
from jina.math.helper import top_k

In [81]:
dists, inds = top_k(sqeuclidean(X,Y), 10)

In [82]:
inds.shape, dists.shape

((10, 10), (10, 10))

In [267]:
dists[0], inds[0]

(array([0.2, 0.1, 0.6]), array([  1, 423, 412]))

In [84]:
def batch_generator(X, batch):
    for i in range(0,len(X), batch):
        yield X[i:i+batch]    

In [244]:
def batch_top_k(X, Y, batch_size, top_k_value):

    y_generator = batch_generator(Y, batch_size)
    top_dists = np.zeros((X.shape[0], top_k_value))
    top_inds  = np.zeros((X.shape[0], top_k_value), dtype=int)
    
    for ybatch in y_generator:
        distances = sqeuclidean(X, ybatch)
        dists, inds = top_k(distances, top_k_value)
        top_dists, top_ids = get_min_dists(dists, inds, top_dists, top_inds,top_k_value)
        
    return top_dists, top_ids

In [356]:
def get_min_dists(dists, inds, top_dists, top_inds, top_k_value):
    all_dists = np.hstack((dists, top_dists))
    all_inds = np.hstack((inds, top_inds))
    best_inds = np.argpartition(all_dists, kth=top_k_value, axis=1)
    d_mat = np.array([all_dists[i,best_inds[i][:top_k_value]] for i in range(len(all_dists))])
    i_mat = np.array([best_inds[i,best_inds[i][:top_k_value]] for i in range(len(all_dists))])
    return all_inds, i_mat

In [358]:
dists = np.array([[0.3,0.1,0.6],
                  [0.9,0.6,0.9]])

inds = np.array([[1,423,412],
                 [5,6,2]]) 

top_dists = np.array([[0.2,0.5,0.9],
                      [0.1,0.5,0.7]])

top_inds = np.array([[5,8,3],
                     [9,10,11]])

top_k_value = 3



In [391]:
all_dists = np.hstack((dists, top_dists))
all_inds = np.hstack((inds, top_inds))
best_inds = np.argpartition(all_dists, kth=top_k_value, axis=1)[:,:top_k_value]

In [393]:
best_inds

array([[1, 3, 0],
       [3, 4, 1]])

In [401]:
all_dists[:,best_inds]

array([[[0.1, 0.2, 0.3],
        [0.2, 0.5, 0.1]],

       [[0.6, 0.1, 0.9],
        [0.1, 0.5, 0.6]]])

In [403]:
?np.searchsorted

In [409]:
# np.searchsorted([0.3,0.1,0.4], [0.7,0.2,0.5], side='left')

In [413]:
all_dists

array([[0.3, 0.1, 0.6, 0.2, 0.5, 0.9],
       [0.9, 0.6, 0.9, 0.1, 0.5, 0.7]])

In [419]:
all_dists[:,best_inds]

array([[[0.1, 0.2, 0.3],
        [0.2, 0.5, 0.1]],

       [[0.6, 0.1, 0.9],
        [0.1, 0.5, 0.6]]])

In [363]:
d_mat = np.array([all_dists[i,best_inds[i][:top_k_value]] for i in range(len(all_dists))])
i_mat = np.array([best_inds[i,best_inds[i][:top_k_value]] for i in range(len(all_dists))])
print(d_mat)
print(i_mat)

[[0.1 0.2 0.3]
 [0.1 0.5 0.6]]
[[3 4 1]
 [5 0 4]]


In [ ]:
dd

In [355]:
get_min_dists(dists, 
              inds, 
              top_dists,
              top_inds,
              top_k_value)

(array([[  1, 423, 412,   5,   8,   3],
        [  5,   6,   2,   9,  10,  11]]),
 array([[3, 4, 1],
        [5, 0, 4]]))